In [23]:
import psycopg2
import requests
import json

以下是在測試初始化DB的過程

In [34]:
def get_api():
    with open('./credentials/youtube_api.json', 'r') as fp:
        token = json.load(fp)['token']
        return token

def get_information(video: str, api_key: str):
    url_video = f'https://www.googleapis.com/youtube/v3/videos?id={video}&key={api_key}&part=snippet'
    response_video = requests.get(url_video)
    response_video = response_video.json()
    
    channelId = (response_video['items'][0]).get('snippet').get('channelId')
    channelTitle = (response_video['items'][0]).get('snippet').get('channelTitle')
    print(channelId, ' ', channelTitle)

    url_channel = f'https://www.googleapis.com/youtube/v3/channels?part=contentDetails&id={channelId}&key={api_key}'
    response_channel = requests.get(url_channel)
    response_channel = response_channel.json()
    

    playListId = response_channel['items'][0].get('contentDetails').get('relatedPlaylists').get('uploads')

    url_playlist = f'https://www.googleapis.com/youtube/v3/playlistItems?part=snippet,contentDetails,status&playlistId={playListId}&key={api_key}&maxResults=20'
    response_playlist = requests.get(url_playlist)
    response_playlist = response_playlist.json()

    videoId = []
    for info in response_playlist['items']:
        video_id = info['contentDetails'].get('videoId')
        videoId.append((channelId, video_id))
    return channelId, channelTitle, playListId, videoId

def insert_channel(channelId, playlistId, channelTitle):
    connection = psycopg2.connect(
                                database = "airflow",
                                user = "airflow",
                                password = "airflow",
                                host = "127.0.0.1",
                                port = '5432'
                            )
    cursor = connection.cursor()
    query = f'''INSERT INTO "channel"(channel_id, playlist_id, channel_title) VALUES(%s, %s, %s)'''
    cursor.execute(query, (channelId, playlistId, channelTitle))
    connection.commit()
    cursor.close()

def insert_video(data):
    connection = psycopg2.connect(
                                database = "airflow",
                                user = "airflow",
                                password = "airflow",
                                host = "127.0.0.1",
                                port = '5432'
                            )
    cursor = connection.cursor()
    query = f'''INSERT INTO video(channel_id, video_id) VALUES(%s, %s)'''
    cursor.executemany(query, data)
    connection.commit()
    cursor.close()




In [33]:
data = ['uLnmTXxpK0Q']
token = get_api()
for video in data:
    channelId, channelTitle, playListId, videoId = get_information(video, token)
    insert_channel(channelId, playListId, channelTitle)
    # insert_video(videoId)

UC8SL1DuWX8YpGe8Y781zq8A   Hey It's Dena


In [16]:
videoId

[('UCA33Z_T-w17Jf-4vnIwRGsQ', 'qxbvNeBwxuI'),
 ('UCA33Z_T-w17Jf-4vnIwRGsQ', '7Qlefvb1260'),
 ('UCA33Z_T-w17Jf-4vnIwRGsQ', '2bUyEcCFrRw'),
 ('UCA33Z_T-w17Jf-4vnIwRGsQ', 'wOLjyZSb3Ig'),
 ('UCA33Z_T-w17Jf-4vnIwRGsQ', 'vSmR5kOKPdY'),
 ('UCA33Z_T-w17Jf-4vnIwRGsQ', 'SIxdvnmIBJ0'),
 ('UCA33Z_T-w17Jf-4vnIwRGsQ', '7jadIik6Pm0'),
 ('UCA33Z_T-w17Jf-4vnIwRGsQ', 'D2qevhlkdHU'),
 ('UCA33Z_T-w17Jf-4vnIwRGsQ', '8vyJuXmrjA0'),
 ('UCA33Z_T-w17Jf-4vnIwRGsQ', 'aundT-ma3tg'),
 ('UCA33Z_T-w17Jf-4vnIwRGsQ', 'E-F3HvX_zQ8'),
 ('UCA33Z_T-w17Jf-4vnIwRGsQ', 'auA7-0YMZuI'),
 ('UCA33Z_T-w17Jf-4vnIwRGsQ', 'Ou1aa9wnsA0'),
 ('UCA33Z_T-w17Jf-4vnIwRGsQ', '9yybcjDqxMM'),
 ('UCA33Z_T-w17Jf-4vnIwRGsQ', 'wWIFbdOLfFM'),
 ('UCA33Z_T-w17Jf-4vnIwRGsQ', 'fRO8mJIRays'),
 ('UCA33Z_T-w17Jf-4vnIwRGsQ', 'zN32UPb2TdI'),
 ('UCA33Z_T-w17Jf-4vnIwRGsQ', 'aInwKp5DAv0'),
 ('UCA33Z_T-w17Jf-4vnIwRGsQ', 'rLhprM2rwFE'),
 ('UCA33Z_T-w17Jf-4vnIwRGsQ', 'JnVzHeoBmyI')]

以下是開發定期查找是否有新影片的過程

In [41]:
connection = psycopg2.connect(
                            database = "airflow",
                            user = "airflow",
                            password = "airflow",
                            host = "127.0.0.1",
                            port = '5432'
                        )
cursor = connection.cursor()
query = f'''SELECT channel_id, playlist_id, channel_title FROM channel;'''
cursor.execute(query)
flag = cursor.fetchall()
cursor.close()
print(flag)    

[('UCWv5gTeR3eEukPgMNBmWJvw', 'UUWv5gTeR3eEukPgMNBmWJvw', 'Chen Lily'), ('UC_LTk4kwxHI0Tj0568u7cxg', 'UU_LTk4kwxHI0Tj0568u7cxg', 'Lillian Chiu'), ('UC8SL1DuWX8YpGe8Y781zq8A', 'UU8SL1DuWX8YpGe8Y781zq8A', "Hey It's Dena")]


In [38]:
if flag:
    print("true")

true


In [42]:
def get_api():
    with open('./credentials/youtube_api.json', 'r') as fp:
        token = json.load(fp)['token']
        return token

def check_video_is_exit(video_id):
    connection = psycopg2.connect(
                            database = "airflow",
                            user = "airflow",
                            password = "airflow",
                            host = "127.0.0.1",
                            port = '5432'
                        )
    cursor = connection.cursor()
    query = f'''SELECT video_id FROM video WHERE video_id = '{video_id}';''' 
    cursor.execute(query)
    if cursor.fetchall():
        return True
    else:
        return False

In [43]:
token = get_api()
video_do_not_store = []
for info in flag:
    channel_id = info[0]
    playlist_id = info[1]
    url_playlist = f'https://www.googleapis.com/youtube/v3/playlistItems?part=snippet,contentDetails,status&playlistId={playlist_id}&key={token}&maxResults=3'
    response_playlist = requests.get(url_playlist)
    response_playlist = response_playlist.json()
    for info in response_playlist['items']:
        video_id = info['contentDetails'].get('videoId')
        if check_video_is_exit(video_id):
            continue
        else:
            video_do_not_store.append((channel_id, video_id))
if len(video_do_not_store) == 0:
    print('do_nothing')
else:
    print('insert DB')

insert DB


In [44]:
video_do_not_store

[('UCWv5gTeR3eEukPgMNBmWJvw', '6b-ZR2rAwEg')]